In [ ]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.externals import joblib
import sys
import json

In [ ]:
import clipper_manager as cl
ec2_host = "ec2-54-67-123-237.us-west-1.compute.amazonaws.com"
user = "ubuntu"
key = os.path.expanduser("~/.ssh/aws_rsa")
clipper = cl.Cluster(ec2_host, user, key)

In [ ]:
clipper.start_clipper()

In [ ]:
clipper.get_app_info("rise_demo")

In [ ]:
candidate_models = [
    {"model_name": "sklearn_cifar", "model_version": 1},
    {"model_name": "tf_cifar", "model_version": 1},
]

clipper.add_app("rise_demo", candidate_models, input_type="doubles", slo_micros=50000)

In [ ]:
clipper.get_app_info("rise_demo")

# Train Scikit-Learn SVM

In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn import svm
from sklearn.externals import joblib

classes = ['airplane', 'automobile', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
positive_class = classes.index('airplane')
negative_class = classes.index('bird')

def load_cifar(cifar_location, cifar_filename = "cifar_train.data", norm=True):
    cifar_path = os.path.join(cifar_location,cifar_filename)
    print("Source file: %s" % cifar_path)
    df = pd.read_csv(cifar_path, sep=",", header=None)
    data = df.values
    print("Number of image files: %d" % len(data))
    y = data[:,0]
    X = data[:,1:]
    Z = X
    if norm:
        mu = np.mean(X,0)
        sigma = np.var(X,0)
        Z = (X - mu) / np.array([np.sqrt(z) if z > 0 else 1. for z in sigma])
    return (Z, y)

def filter_data(X, y):
    X_train, y_train = [], []
    for (example, label) in zip(X, y):
        if label == positive_class:
            X_train.append(example)
            y_train.append(1.0)
        elif label == negative_class:
            X_train.append(example)
            y_train.append(-1.0)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    return X_train, y_train


## Load CIFAR-10 Data

In [ ]:
cifar_path = os.path.expanduser("~/model-serving/data/cifar")
X, y = load_cifar(cifar_path)
X_train, y_train = filter_data(X, y)

X_test, y_test = load_cifar(cifar_path, 'cifar_test.data')
X_test, y_test = filter_data(X_test, y_test)

## Train and score model

In [ ]:
def train_model(m):
    m.fit(X_train, y_train)
    print(m.score(X_train[:1000], y_train[:1000]))
    print(m.score(X_test, y_test))
    return m
    
lin_model = train_model(lm.LogisticRegression())
# svm_model = train_model(svm.LinearSVC())

## Deploy Scikit-Learn Model

In [ ]:
model_added = clipper.add_sklearn_model(
    "sklearn_cifar",
    1,
    lin_model,
    "clipper/sklearn_cifar_container",
    ["cifar", "sklearn"],
    input_type="doubles",
    num_containers=1
)

# model_added = clipper.add_sklearn_model(
#     "sklearn_cifar",
#     1,
#     "/tmp/sklearn_cifar",
#     "clipper/sklearn_cifar_container",
#     ["cifar", "sklearn"],
#     "floats"
# )
print model_added

In [ ]:
clipper.add_container("sklearn_cifar", 1)

# Clean Up

In [ ]:
# clipper.stop_all()
clipper.cleanup()